In [2]:
import sys #access to system parameters https://docs.python.org/3/library/sys.html
print("Python version: {}". format(sys.version))

import pandas as pd #collection of functions for data processing and analysis modeled after R dataframes with SQL like features
print("pandas version: {}". format(pd.__version__))

import matplotlib #collection of functions for scientific and publication-ready visualization
import matplotlib as plt
print("matplotlib version: {}". format(matplotlib.__version__))

import numpy as np #foundational package for scientific computing
print("NumPy version: {}". format(np.__version__))

import scipy as sp #collection of functions for scientific computing and advance mathematics
print("SciPy version: {}". format(sp.__version__)) 

import IPython
from IPython import display #pretty printing of dataframes in Jupyter notebook
print("IPython version: {}". format(IPython.__version__)) 

import sklearn #collection of machine learning algorithms
print("scikit-learn version: {}". format(sklearn.__version__))

#misc libraries
import random
import time
#ignore warnings
import warnings
warnings.filterwarnings('ignore')
print('-'*25)
import re
import os
print('\n'.join(os.listdir('../Downloads')))
import matplotlib.pyplot as plt
%matplotlib inline

Python version: 3.9.7 (v3.9.7:1016ef3790, Aug 30 2021, 16:39:15) 
[Clang 6.0 (clang-600.0.57)]
pandas version: 1.3.4
matplotlib version: 3.4.3
NumPy version: 1.21.3
SciPy version: 1.7.1
IPython version: 7.29.0
scikit-learn version: 1.0.1
-------------------------
ass2.pages
hw2
csresume_sample-converted.docx
Slack-4.19.0-macOS.dmg
2019_manhattan.xlsx
saketh_resume1 (2).pdf
sparksql-commands (1).txt
_saketh_teaching_assistantSteinhardt_CoverLetter.pdf
Assignment2 (1).pdf
_saketh_researchassistant_CoverLetter.pdf
productmanagement_sakethReddy.pdf
Assignment1-Map-Reduce (2).pdf
ProgrammingAssignment1_BrunoMacedoPasini (2)
image (5).png
http-ethereal-trace-2 (1)
sublime_text_build_4113_mac.zip
VMware-Horizon-Client-2106-8.3.0-18251973.dmg
Remove background (1).png
ProgrammingAssignment1_BrunoMacedoPasini (2).zip
spark-commands (1).txt
work.pdf
VirtualBox-6.1.26-145957-OSX.dmg
psr.jpg
AidDataCore_ResearchRelease_Level1_v3.1.zip
aiddata-countries-only - aiddata-countries-only (1).csv
Saketh-

In [3]:
url = 'https://data.cityofnewyork.us/api/views/qgea-i56i/rows.csv?accessType=DOWNLOAD'

In [4]:
raw_data = pd.read_csv(url)
raw_data

,CMPLNT_NUM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,ADDR_PCT_CD,RPT_DT,KY_CD,OFNS_DESC,PD_CD,...,SUSP_SEX,TRANSIT_DISTRICT,Latitude,Longitude,Lat_Lon,PATROL_BORO,STATION_NAME,VIC_AGE_GROUP,VIC_RACE,VIC_SEX
0,394506329,12/31/2019,17:30:00,NaN,NaN,32.0,12/31/2019,118,DANGEROUS WEAPONS,793.0,...,NaN,NaN,40.820927,-73.943324,"(40.82092679700002, -73.94332421899996)",PATROL BORO MAN NORTH,NaN,UNKNOWN,UNKNOWN,E
1,968873685,12/29/2019,16:31:00,12/29/2019,16:54:00,47.0,12/29/2019,113,FORGERY,729.0,...,NaN,NaN,40.885701,-73.861640,"(40.885701406000074, -73.86164032499995)",PATROL BORO BRONX,NaN,UNKNOWN,UNKNOWN,E
2,509837549,12/15/2019,18:45:00,NaN,NaN,109.0,12/29/2019,578,HARRASSMENT 2,638.0,...,M,NaN,40.742281,-73.819824,"(40.74228115600005, -73.81982408)",PATROL BORO QUEENS NORTH,NaN,25-44,WHITE HISPANIC,F
3,352454313,12/28/2019,01:00:00,NaN,NaN,47.0,12/28/2019,126,MISCELLANEOUS PENAL LAW,117.0,...,M,NaN,40.875311,-73.847545,"(40.87531145100007, -73.84754521099995)",PATROL BORO BRONX,NaN,UNKNOWN,UNKNOWN,E
4,248803469,09/05/2008,21:41:00,NaN,NaN,NaN,09/05/2008,101,MURDER & NON-NEGL. MANSLAUGHTER,NaN,...,NaN,NaN,40.698827,-73.938819,"(40.698827283, -73.938819047)",NaN,NaN,25-44,BLACK,M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7375988,744350707,12/17/2018,03:00:00,12/17/2018,03:05:00,47.0,12/17/2018,348,VEHICLE AND TRAFFIC LAWS,916.0,...,NaN,NaN,40.887333,-73.847250,"(40.887332818, -73.847250013)",PATROL BORO BRONX,NaN,25-44,WHITE HISPANIC,F
7375989,380927470,06/05/2018,23:30:00,06/06/2018,11:00:00,43.0,06/07/2018,110,GRAND LARCENY OF MOTOR VEHICLE,450.0,...,M,NaN,40.827414,-73.877946,"(40.827414051, -73.877945775)",PATROL BORO BRONX,NaN,25-44,BLACK,M
7375990,502083804,02/10/2018,08:30:00,02/10/2018,08:45:00,42.0,02/10/2018,341,PETIT LARCENY,339.0,...,U,NaN,40.834528,-73.903836,"(40.834528146, -73.903836069)",PATROL BORO BRONX,NaN,45-64,BLACK,F
7375991,609916130,12/11/2018,07:30:00,12/11/2018,07:50:00,32.0,12/11/2018,344,ASSAULT 3 & RELATED OFFENSES,101.0,...,U,NaN,40.811812,-73.938329,"(40.81181176, -73.938328651)",PATROL BORO MAN NORTH,NaN,<18,BLACK,M


In [1]:
raw_data.head(50)

NameError: name 'raw_data' is not defined

In [7]:
raw_data.dtypes

CMPLNT_NUM             int64
CMPLNT_FR_DT          object
CMPLNT_FR_TM          object
CMPLNT_TO_DT          object
CMPLNT_TO_TM          object
ADDR_PCT_CD          float64
RPT_DT                object
KY_CD                  int64
OFNS_DESC             object
PD_CD                float64
PD_DESC               object
CRM_ATPT_CPTD_CD      object
LAW_CAT_CD            object
BORO_NM               object
LOC_OF_OCCUR_DESC     object
PREM_TYP_DESC         object
JURIS_DESC            object
JURISDICTION_CODE    float64
PARKS_NM              object
HADEVELOPT            object
HOUSING_PSA           object
X_COORD_CD           float64
Y_COORD_CD           float64
SUSP_AGE_GROUP        object
SUSP_RACE             object
SUSP_SEX              object
TRANSIT_DISTRICT     float64
Latitude             float64
Longitude            float64
Lat_Lon               object
PATROL_BORO           object
STATION_NAME          object
VIC_AGE_GROUP         object
VIC_RACE              object
VIC_SEX       

In [9]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7375993 entries, 0 to 7375992
Data columns (total 35 columns):
 #   Column             Dtype  
---  ------             -----  
 0   CMPLNT_NUM         int64  
 1   CMPLNT_FR_DT       object 
 2   CMPLNT_FR_TM       object 
 3   CMPLNT_TO_DT       object 
 4   CMPLNT_TO_TM       object 
 5   ADDR_PCT_CD        float64
 6   RPT_DT             object 
 7   KY_CD              int64  
 8   OFNS_DESC          object 
 9   PD_CD              float64
 10  PD_DESC            object 
 11  CRM_ATPT_CPTD_CD   object 
 12  LAW_CAT_CD         object 
 13  BORO_NM            object 
 14  LOC_OF_OCCUR_DESC  object 
 15  PREM_TYP_DESC      object 
 16  JURIS_DESC         object 
 17  JURISDICTION_CODE  float64
 18  PARKS_NM           object 
 19  HADEVELOPT         object 
 20  HOUSING_PSA        object 
 21  X_COORD_CD         float64
 22  Y_COORD_CD         float64
 23  SUSP_AGE_GROUP     object 
 24  SUSP_RACE          object 
 25  SUSP_SEX          

Data types and data has been loaded and reviewed